# Module 03: Azure ML Studio Introduction

**Difficulty**: ⭐⭐
**Estimated Time**: 55 minutes
**Prerequisites**: 
- [Module 01: AWS SageMaker Basics](01_aws_sagemaker_basics.ipynb)
- Basic understanding of machine learning workflows
- Familiarity with Python and scikit-learn

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand Azure ML workspace architecture and components
2. Configure and manage compute instances and clusters
3. Use Azure ML Designer for visual ML pipeline creation
4. Train models using Azure ML SDK (v2)
5. Manage datasets and datastores effectively
6. Apply Azure AutoML for automated model selection

## 1. Setup and Introduction

### Azure ML Studio Overview

Azure ML Studio is Microsoft's cloud platform for the complete ML lifecycle.

```
┌─────────────────────────────────────────────────────────┐
│           Azure ML Workspace Architecture                │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  ┌─────────────────────────────────────────────────┐   │
│  │         Azure ML Workspace                      │   │
│  ├─────────────────────────────────────────────────┤   │
│  │                                                 │   │
│  │  Compute Resources    Datastores   Models      │   │
│  │  ┌──────────────┐    ┌─────────┐  ┌────────┐  │   │
│  │  │ Instances    │    │ Blob    │  │Registry│  │   │
│  │  │ Clusters     │    │ ADLS    │  │Versions│  │   │
│  │  │ Inference    │    │ SQL DB  │  └────────┘  │   │
│  │  └──────────────┘    └─────────┘              │   │
│  │                                                 │   │
│  │  Experiments      Pipelines      Endpoints     │   │
│  │  ┌──────────┐    ┌──────────┐   ┌─────────┐  │   │
│  │  │ Runs     │    │ Designer │   │ Online  │  │   │
│  │  │ Metrics  │    │ SDK      │   │ Batch   │  │   │
│  │  │ Logs     │    │ AutoML   │   └─────────┘  │   │
│  │  └──────────┘    └──────────┘                 │   │
│  └─────────────────────────────────────────────────┘   │
│                                                         │
│  Integrated with:                                       │
│  - Azure DevOps (CI/CD)                                 │
│  - GitHub Actions                                       │
│  - MLflow                                               │
│  - Power BI                                             │
└─────────────────────────────────────────────────────────┘
```

**Cost Note**: Azure offers a free tier with limited compute hours. Always stop compute instances when not in use!

In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Mock Azure ML SDK for demonstration
# In production, use: from azure.ai.ml import MLClient

class MockAzureMLWorkspace:
    """Simulates Azure ML Workspace for educational purposes"""
    
    def __init__(self, name, subscription_id, resource_group):
        self.name = name
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.compute_targets = {}
        self.datastores = {}
        self.experiments = {}
    
    def get_details(self):
        """Get workspace configuration details"""
        return {
            'name': self.name,
            'subscription_id': self.subscription_id,
            'resource_group': self.resource_group,
            'location': 'eastus',
            'workspace_id': f'ws-{np.random.randint(1000, 9999)}'
        }

## 2. Azure ML Workspace Setup

The workspace is the top-level resource for Azure ML. It contains all your ML assets.

### Workspace Components

```
Workspace
├── Compute
│   ├── Compute Instances (development)
│   ├── Compute Clusters (training)
│   ├── Inference Clusters (deployment)
│   └── Attached Compute (external)
├── Data
│   ├── Datastores (connections)
│   └── Datasets (versioned data)
├── Assets
│   ├── Models (registered models)
│   ├── Environments (dependencies)
│   └── Components (reusable code)
└── Jobs
    ├── Experiments (training runs)
    ├── Pipelines (workflows)
    └── Endpoints (deployments)
```

In [ ]:
# Simulate workspace creation

def create_workspace_config():
    """Define Azure ML workspace configuration"""
    config = {
        'workspace_name': 'my-ml-workspace',
        'subscription_id': 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx',
        'resource_group': 'ml-resources',
        'location': 'eastus',
        'storage_account': 'mlstorageaccount',
        'key_vault': 'mlkeyvault',
        'app_insights': 'mlappinsights',
        'container_registry': 'mlcontainerregistry'
    }
    return config

ws_config = create_workspace_config()
print("Azure ML Workspace Configuration:")
print(json.dumps(ws_config, indent=2))

# Create mock workspace
workspace = MockAzureMLWorkspace(
    name=ws_config['workspace_name'],
    subscription_id=ws_config['subscription_id'],
    resource_group=ws_config['resource_group']
)

print("\nWorkspace created successfully!")
print(f"Workspace ID: {workspace.get_details()['workspace_id']}")

In [ ]:
# Connection string for workspace

def generate_workspace_config_json(workspace_details):
    """
    Generate config.json for workspace connection
    This file allows SDK to connect to your workspace
    """
    config_json = {
        "subscription_id": workspace_details['subscription_id'],
        "resource_group": workspace_details['resource_group'],
        "workspace_name": workspace_details['name']
    }
    
    print("Save this as config.json in your project directory:")
    print(json.dumps(config_json, indent=2))
    print("\nThen connect with: MLClient.from_config()")
    return config_json

config_json = generate_workspace_config_json(workspace.get_details())

## 3. Compute Resources

Azure ML provides several compute options for different scenarios.

### Compute Types Comparison

```
┌─────────────────────────────────────────────────────────┐
│              Azure ML Compute Types                      │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  1. Compute Instance                                    │
│     Use: Development, notebooks, debugging              │
│     Features: Jupyter, VS Code, RStudio                 │
│     Scaling: Single VM, can stop/start                  │
│     Cost: ~$0.15-$5/hour (varies by size)               │
│                                                         │
│  2. Compute Cluster                                     │
│     Use: Training jobs, batch inference                 │
│     Features: Auto-scaling (0 to N nodes)               │
│     Scaling: Multi-node, scales to zero                 │
│     Cost: Pay only when running jobs                    │
│                                                         │
│  3. Inference Cluster (AKS)                             │
│     Use: Production deployments                         │
│     Features: High availability, auto-scaling           │
│     Scaling: Kubernetes-based                           │
│     Cost: ~$0.10/hour + node costs                      │
│                                                         │
│  4. Attached Compute                                    │
│     Use: Existing VMs, Databricks, Synapse              │
│     Features: Use external compute                      │
│     Scaling: Managed externally                         │
│     Cost: Based on attached resource                    │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Compute instance configuration

def create_compute_instance_config():
    """Configure compute instance for development"""
    config = {
        'name': 'dev-instance-01',
        'vm_size': 'STANDARD_DS3_V2',  # 4 cores, 14GB RAM
        'ssh_public_access_enabled': False,
        'idle_time_before_shutdown_minutes': 30,  # Auto-shutdown
        'applications': [
            'jupyter',
            'jupyterlab',
            'rstudio',
            'vscode'
        ]
    }
    return config

instance_config = create_compute_instance_config()
print("Compute Instance Configuration:")
print(json.dumps(instance_config, indent=2))
print("\n💡 Tip: Set idle shutdown to save costs!")

In [ ]:
# Compute cluster configuration

def create_compute_cluster_config():
    """Configure auto-scaling compute cluster for training"""
    config = {
        'name': 'training-cluster',
        'vm_size': 'STANDARD_DS3_V2',
        'min_instances': 0,  # Scales to zero when idle
        'max_instances': 4,  # Maximum concurrent nodes
        'idle_seconds_before_scaledown': 300,  # 5 minutes
        'tier': 'dedicated',  # or 'low_priority' for cost savings
        'location': 'eastus'
    }
    
    print("💰 Cost Optimization:")
    print("- Min instances = 0: No cost when idle")
    print("- Low priority VMs: Up to 80% cost savings")
    print("- Auto-scaledown: Minimizes billable hours\n")
    
    return config

cluster_config = create_compute_cluster_config()
print("Compute Cluster Configuration:")
print(json.dumps(cluster_config, indent=2))

In [ ]:
# Simulate compute cluster scaling

class ComputeClusterSimulator:
    """Simulates Azure ML compute cluster auto-scaling"""
    
    def __init__(self, min_nodes=0, max_nodes=4):
        self.min_nodes = min_nodes
        self.max_nodes = max_nodes
        self.current_nodes = min_nodes
        self.jobs_queue = []
    
    def submit_job(self, job_name):
        """Submit training job to cluster"""
        self.jobs_queue.append(job_name)
        self._scale_up()
        print(f"Job '{job_name}' submitted. Nodes: {self.current_nodes}")
    
    def _scale_up(self):
        """Scale up cluster based on queue length"""
        needed_nodes = min(len(self.jobs_queue), self.max_nodes)
        if needed_nodes > self.current_nodes:
            self.current_nodes = needed_nodes
            print(f"⬆️  Scaling up to {self.current_nodes} nodes")
    
    def complete_job(self):
        """Complete a job and potentially scale down"""
        if self.jobs_queue:
            completed = self.jobs_queue.pop(0)
            print(f"✓ Job '{completed}' completed")
            if not self.jobs_queue:
                self._scale_down()
    
    def _scale_down(self):
        """Scale down to minimum when idle"""
        self.current_nodes = self.min_nodes
        print(f"⬇️  Scaling down to {self.current_nodes} nodes (idle)")

In [ ]:
# Test cluster scaling
cluster = ComputeClusterSimulator(min_nodes=0, max_nodes=4)

# Submit multiple jobs
print("Submitting jobs:\n")
cluster.submit_job('train-model-v1')
cluster.submit_job('train-model-v2')
cluster.submit_job('hyperparameter-tuning')

print("\nCompleting jobs:\n")
cluster.complete_job()
cluster.complete_job()
cluster.complete_job()

print(f"\nFinal state: {cluster.current_nodes} nodes (saves cost when at 0)")

## 4. Azure ML Designer (Visual ML)

Designer provides a drag-and-drop interface for building ML pipelines without code.

### Designer Pipeline Architecture

```
┌─────────────────────────────────────────────────────────┐
│         Azure ML Designer Pipeline                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  ┌──────────┐                                           │
│  │  Data    │                                           │
│  │  Import  │                                           │
│  └────┬─────┘                                           │
│       │                                                 │
│       ▼                                                 │
│  ┌──────────────┐     ┌──────────────┐                 │
│  │ Clean        │────▶│ Normalize    │                 │
│  │ Missing Data │     │ Data         │                 │
│  └──────────────┘     └──────┬───────┘                 │
│                              │                          │
│                              ▼                          │
│                       ┌──────────────┐                 │
│                       │ Split Data   │                 │
│                       │ (70/30)      │                 │
│                       └──┬──────┬────┘                 │
│                          │      │                      │
│                  Train   │      │   Test               │
│                          ▼      ▼                      │
│                    ┌─────────┐  │                      │
│                    │ Train   │  │                      │
│                    │ Model   │  │                      │
│                    └────┬────┘  │                      │
│                         │       │                      │
│                         ▼       ▼                      │
│                       ┌───────────┐                    │
│                       │ Score     │                    │
│                       │ Model     │                    │
│                       └─────┬─────┘                    │
│                             │                          │
│                             ▼                          │
│                       ┌───────────┐                    │
│                       │ Evaluate  │                    │
│                       │ Model     │                    │
│                       └───────────┘                    │
└─────────────────────────────────────────────────────────┘
```

**Designer Benefits**:
- No code required
- Visual pipeline creation
- Reusable components
- Built-in algorithms
- Easy experimentation

In [ ]:
# Simulate Designer pipeline components

class DesignerPipeline:
    """Simulates Azure ML Designer pipeline execution"""
    
    def __init__(self, name):
        self.name = name
        self.components = []
        self.data = None
    
    def add_component(self, component_type, params):
        """Add component to pipeline"""
        component = {
            'type': component_type,
            'params': params,
            'order': len(self.components)
        }
        self.components.append(component)
        print(f"Added: {component_type}")
    
    def run_pipeline(self, input_data):
        """Execute pipeline on data"""
        print(f"\nRunning pipeline: {self.name}")
        print(f"Components: {len(self.components)}\n")
        
        self.data = input_data
        for comp in self.components:
            print(f"Executing: {comp['type']}...")
        
        print("\n✓ Pipeline completed successfully!")
        return self.data

In [ ]:
# Build a Designer pipeline
pipeline = DesignerPipeline('credit-risk-model')

# Add components in order
pipeline.add_component('Import Data', {'source': 'Azure Blob Storage'})
pipeline.add_component('Clean Missing Data', {'method': 'mean'})
pipeline.add_component('Normalize Data', {'method': 'MinMax'})
pipeline.add_component('Split Data', {'ratio': 0.7})
pipeline.add_component('Train Model', {'algorithm': 'Logistic Regression'})
pipeline.add_component('Score Model', {})
pipeline.add_component('Evaluate Model', {'metrics': ['AUC', 'Accuracy']})

# Run pipeline
sample_data = pd.DataFrame(np.random.randn(100, 5))
result = pipeline.run_pipeline(sample_data)

## 5. Training with Azure ML SDK (v2)

The Azure ML SDK provides programmatic access to all workspace features.

### SDK Training Workflow

```
1. Define Environment (dependencies)
2. Create Training Script
3. Configure Compute Target
4. Submit Training Job
5. Monitor Progress
6. Register Model
```

In [ ]:
# Define training environment

def create_training_environment():
    """Define conda environment for training job"""
    env_config = {
        'name': 'sklearn-env',
        'version': '1',
        'conda_specification': {
            'name': 'sklearn-env',
            'channels': ['conda-forge'],
            'dependencies': [
                'python=3.9',
                'pip',
                {
                    'pip': [
                        'scikit-learn==1.3.0',
                        'pandas==2.0.0',
                        'azureml-mlflow',
                        'numpy'
                    ]
                }
            ]
        },
        'docker_image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04'
    }
    return env_config

env = create_training_environment()
print("Training Environment:")
print(json.dumps(env, indent=2))

In [ ]:
# Create training job configuration

def create_training_job_config():
    """Configure training job with compute and parameters"""
    job_config = {
        'code': './src',  # Training script directory
        'command': 'python train.py --epochs 10 --lr 0.001',
        'environment': 'sklearn-env:1',
        'compute': 'training-cluster',
        'experiment_name': 'classification-experiment',
        'display_name': 'sklearn-training-run-001',
        'inputs': {
            'training_data': {
                'type': 'uri_folder',
                'path': 'azureml://datastores/workspaceblobstore/paths/data/'
            }
        },
        'outputs': {
            'model_output': {
                'type': 'uri_folder',
                'path': 'azureml://datastores/workspaceblobstore/paths/models/'
            }
        }
    }
    return job_config

job_config = create_training_job_config()
print("Training Job Configuration:")
print(json.dumps(job_config, indent=2))

In [ ]:
# Simulate training job execution

class TrainingJobSimulator:
    """Simulates Azure ML training job execution"""
    
    def __init__(self, job_name):
        self.job_name = job_name
        self.status = 'NotStarted'
        self.metrics = {}
    
    def submit(self):
        """Submit training job to compute cluster"""
        self.status = 'Running'
        print(f"Job '{self.job_name}' submitted")
        print(f"Status: {self.status}")
        print("Monitor at: https://ml.azure.com/runs/{job_id}\n")
    
    def log_metrics(self, epoch, metrics):
        """Log training metrics (simulated)"""
        self.metrics[epoch] = metrics
        print(f"Epoch {epoch}: {metrics}")
    
    def complete(self):
        """Mark job as completed"""
        self.status = 'Completed'
        print(f"\nJob '{self.job_name}' completed successfully!")
        print(f"Final metrics: {self.metrics[max(self.metrics.keys())]}")

In [ ]:
# Run simulated training job
job = TrainingJobSimulator('sklearn-classification-001')
job.submit()

# Simulate training epochs
for epoch in range(1, 6):
    metrics = {
        'loss': 1.0 / (epoch + 1) + np.random.rand() * 0.1,
        'accuracy': 0.6 + (epoch * 0.05) + np.random.rand() * 0.02
    }
    job.log_metrics(epoch, metrics)

job.complete()

## 6. Dataset and Datastore Management

Datastores connect to storage services, while Datasets provide versioned data abstractions.

### Data Management Architecture

```
┌─────────────────────────────────────────────────────────┐
│         Azure ML Data Management                         │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Datastores (Connections)                               │
│  ┌─────────────────────────────────────────┐           │
│  │ Azure Blob Storage                      │           │
│  │ Azure Data Lake Storage Gen2            │           │
│  │ Azure SQL Database                      │           │
│  │ Azure PostgreSQL                        │           │
│  └─────────────────────────────────────────┘           │
│                     │                                   │
│                     ▼                                   │
│  Datasets (Versioned Data)                              │
│  ┌─────────────────────────────────────────┐           │
│  │ FileDataset (files/folders)             │           │
│  │ - Images, text, any file format         │           │
│  │                                         │           │
│  │ TabularDataset (structured)             │           │
│  │ - CSV, Parquet, JSON                    │           │
│  │ - SQL query results                     │           │
│  └─────────────────────────────────────────┘           │
│                     │                                   │
│                     ▼                                   │
│  Data Versioning & Lineage                              │
│  - Track data changes                                   │
│  - Reproduce experiments                                │
│  - Monitor data drift                                   │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# Create datastore configuration

def create_datastore_config():
    """Configure connection to Azure Blob Storage"""
    datastore_config = {
        'name': 'ml_datastore',
        'type': 'AzureBlobStorage',
        'account_name': 'mlstorageaccount',
        'container_name': 'mldata',
        'credentials': {
            'type': 'account_key',  # or SAS token, service principal
            'account_key': '[STORED_IN_KEY_VAULT]'
        },
        'protocol': 'https'
    }
    
    print("Security Best Practice:")
    print("- Store credentials in Azure Key Vault")
    print("- Use managed identity when possible")
    print("- Rotate keys regularly\n")
    
    return datastore_config

datastore = create_datastore_config()
print("Datastore Configuration:")
print(json.dumps(datastore, indent=2))

In [ ]:
# Create dataset from datastore

def create_tabular_dataset():
    """Create versioned tabular dataset from CSV files"""
    dataset_config = {
        'name': 'customer_churn_data',
        'version': 1,
        'description': 'Customer churn prediction dataset',
        'type': 'TabularDataset',
        'path': [
            'azureml://datastores/ml_datastore/paths/data/churn/*.csv'
        ],
        'tags': {
            'department': 'marketing',
            'project': 'churn-prediction',
            'data_quality': 'validated'
        },
        'schema': {
            'customer_id': 'string',
            'age': 'integer',
            'tenure': 'integer',
            'monthly_charges': 'float',
            'churn': 'boolean'
        }
    }
    return dataset_config

dataset = create_tabular_dataset()
print("Dataset Configuration:")
print(json.dumps(dataset, indent=2))

## 7. Azure AutoML

AutoML automatically tries multiple algorithms and hyperparameters to find the best model.

### AutoML Process

```
┌─────────────────────────────────────────────────────────┐
│            Azure AutoML Workflow                         │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Input: Dataset + Task Type + Constraints               │
│  ┌─────────────────────────────────────────┐           │
│  │ Task: Classification/Regression/...     │           │
│  │ Time limit: 1 hour                      │           │
│  │ Metric: AUC / RMSE / ...                │           │
│  └─────────────────────────────────────────┘           │
│                     │                                   │
│                     ▼                                   │
│  AutoML Engine                                          │
│  ┌─────────────────────────────────────────┐           │
│  │ 1. Data Preprocessing                   │           │
│  │    - Feature engineering                │           │
│  │    - Scaling, encoding                  │           │
│  │                                         │           │
│  │ 2. Algorithm Selection                  │           │
│  │    - Try multiple algorithms            │           │
│  │    - Ensemble methods                   │           │
│  │                                         │           │
│  │ 3. Hyperparameter Tuning                │           │
│  │    - Bayesian optimization              │           │
│  │    - Random search                      │           │
│  └─────────────────────────────────────────┘           │
│                     │                                   │
│                     ▼                                   │
│  Output: Best Model + Explanation                       │
│  ┌─────────────────────────────────────────┐           │
│  │ - Model artifacts                       │           │
│  │ - Feature importance                    │           │
│  │ - Performance metrics                   │           │
│  │ - Deployment-ready                      │           │
│  └─────────────────────────────────────────┘           │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# AutoML configuration

def create_automl_config():
    """Configure AutoML experiment for classification"""
    automl_config = {
        'task': 'classification',
        'primary_metric': 'AUC_weighted',
        'training_data': 'customer_churn_data:1',
        'target_column_name': 'churn',
        'compute_target': 'training-cluster',
        'limits': {
            'timeout_minutes': 60,
            'trial_timeout_minutes': 10,
            'max_trials': 20,
            'max_concurrent_trials': 4,
            'enable_early_termination': True
        },
        'featurization': {
            'mode': 'auto',  # Automatic feature engineering
            'blocked_transformers': [],  # Optionally block transformers
        },
        'validation': {
            'n_cross_validations': 5
        },
        'training': {
            'enable_onnx_compatible_models': True,  # For deployment
            'enable_voting_ensemble': True,
            'enable_stack_ensemble': True
        }
    }
    return automl_config

automl = create_automl_config()
print("AutoML Configuration:")
print(json.dumps(automl, indent=2))

In [ ]:
# Simulate AutoML experiment

class AutoMLSimulator:
    """Simulates Azure AutoML experiment execution"""
    
    def __init__(self, max_trials=10):
        self.max_trials = max_trials
        self.trials = []
        self.algorithms = [
            'LogisticRegression',
            'RandomForest',
            'XGBoost',
            'LightGBM',
            'VotingEnsemble'
        ]
    
    def run(self):
        """Run AutoML experiment with multiple trials"""
        print("Starting AutoML experiment...\n")
        
        for i in range(self.max_trials):
            algorithm = np.random.choice(self.algorithms)
            auc = 0.65 + np.random.rand() * 0.3  # Simulated AUC
            
            trial = {
                'trial': i + 1,
                'algorithm': algorithm,
                'auc': auc,
                'duration': np.random.randint(30, 300)  # seconds
            }
            self.trials.append(trial)
            
            print(f"Trial {i+1}/{self.max_trials}: {algorithm} - AUC: {auc:.4f}")
        
        # Find best model
        best_trial = max(self.trials, key=lambda x: x['auc'])
        print(f"\n✓ Best model: {best_trial['algorithm']}")
        print(f"  AUC: {best_trial['auc']:.4f}")
        
        return best_trial

In [ ]:
# Run AutoML simulation
automl_experiment = AutoMLSimulator(max_trials=10)
best_model = automl_experiment.run()

print("\nAutoML Benefits:")
print("- Tries multiple algorithms automatically")
print("- Handles feature engineering")
print("- Provides model explanations")
print("- Faster than manual tuning")

## Exercises

### Exercise 1: Compute Cost Optimization

You have three training workloads:
1. Daily model retraining (30 min/day)
2. Hyperparameter tuning (4 hours/week)
3. Ad-hoc experiments (variable)

Design a compute strategy using instances and clusters. Calculate monthly costs.

In [ ]:
# Your solution here
def design_compute_strategy():
    """
    Design optimal compute strategy for different workloads
    
    Consider:
    - Compute instance vs cluster
    - VM sizes
    - Auto-shutdown/scale-down
    - Low-priority VMs
    
    Returns:
        dict with strategy and cost estimate
    """
    # TODO: Implement strategy
    pass

# Test your strategy
# strategy = design_compute_strategy()
# print(json.dumps(strategy, indent=2))

### Exercise 2: Pipeline Design

Design a Designer pipeline for a regression task:
- Input: House prices CSV
- Steps: Clean data, engineer features, train model, evaluate
- Output: Trained model and metrics

List all components in order.

In [ ]:
# Your solution here
def design_regression_pipeline():
    """
    Design complete Designer pipeline for house price prediction
    
    Include:
    - Data import and cleaning
    - Feature engineering
    - Model training
    - Evaluation
    
    Returns:
        list of pipeline components
    """
    # TODO: Design pipeline
    pass

# Test your pipeline
# pipeline = design_regression_pipeline()
# for i, component in enumerate(pipeline, 1):
#     print(f"{i}. {component}")

### Exercise 3: Dataset Versioning Strategy

You need to version datasets that:
- Update weekly with new data
- May have schema changes
- Need to be reproducible for compliance

Design a versioning and tagging strategy.

In [ ]:
# Your solution here
def design_versioning_strategy():
    """
    Design dataset versioning strategy
    
    Consider:
    - Version numbering scheme
    - Tags for metadata
    - Schema validation
    - Retention policy
    
    Returns:
        dict with versioning guidelines
    """
    # TODO: Design strategy
    pass

# Test your strategy
# strategy = design_versioning_strategy()
# print(json.dumps(strategy, indent=2))

## Summary

In this notebook, you learned:

1. **Workspace Architecture**: Understanding Azure ML workspace components
2. **Compute Management**: Instances vs clusters, auto-scaling strategies
3. **Designer**: Visual ML pipeline creation without code
4. **SDK Training**: Programmatic job submission and monitoring
5. **Data Management**: Datastores and versioned datasets
6. **AutoML**: Automated model selection and tuning

### Key Takeaways

- Use compute instances for development, clusters for training
- Enable auto-shutdown and scale-to-zero to save costs
- Designer is great for no-code ML workflows
- SDK provides full programmatic control
- Version datasets for reproducibility
- AutoML accelerates model development

### What's Next?

- [Module 04: Azure ML Pipelines and Deployment](04_azure_ml_pipelines_and_deployment.ipynb)
- Explore MLflow integration
- Practice with real Azure ML workspace (free tier)
- Build reusable pipeline components

### Additional Resources

- [Azure ML Documentation](https://docs.microsoft.com/azure/machine-learning/)
- [Azure ML SDK v2](https://docs.microsoft.com/python/api/overview/azure/ml/)
- [Designer Component Reference](https://docs.microsoft.com/azure/machine-learning/component-reference/)
- [AutoML Guide](https://docs.microsoft.com/azure/machine-learning/concept-automated-ml)